In [1]:
import pandas as pd
import polars as pl
import os
from functions.fix_parquet_with_header_data import fix_parquet_with_header_data
from functions.prepare_electric_filenames_from_household_sensors import prepare_electric_filenames_from_household_sensors
from functions.aggregate_data_daily import aggregate_data_daily
from functions.prepare_electric_filenames_from_appliance import prepare_electric_filenames_from_appliance
from pathlib import Path

### Przygotowanie df-a z lokalizacjami plików do agregowania

In [2]:
folder_path = "../../Data/Parquet/household_sensors/sensordata"
household_sensors_data_to_process = prepare_electric_filenames_from_household_sensors(folder_path)

In [3]:
folder_path = "../../Data/Parquet/room_and_appliance_sensors/sensordata"
appliance_data_to_process = prepare_electric_filenames_from_appliance(folder_path)

In [4]:
len(household_sensors_data_to_process["home_id"].unique())

254

In [5]:
appliance_data_to_process

,home_id,electric,appliance_name,filename
0,61,electric-appliance,kettle,home61_kitchen695_sensor1967_electric-applianc...
1,61,electric-appliance,dishwasher,home61_kitchen695_sensor1969_electric-applianc...
2,61,electric-appliance,microwave,home61_kitchen695_sensor1971_electric-applianc...
3,62,electric-appliance,fridgefreezer,home62_kitchen710_sensor1779_electric-applianc...
4,62,electric-appliance,kettle,home62_kitchen710_sensor1780_electric-applianc...
...,...,...,...,...
214,328,electric-appliance,vacuumcleaner,home328_hall2976_sensor31009_electric-applianc...
215,328,electric-appliance,kettle,home328_kitchen2980_sensor31003_electric-appli...
216,328,electric-appliance,toaster,home328_kitchen2980_sensor31004_electric-appli...
217,328,electric-appliance,fridgefreezer,home328_kitchen2980_sensor31005_electric-appli...


In [44]:
household_sensors_data_to_process

,home_id,electric,subcircuit,filename
0,47,electric-mains_electric-combined,None,home47_hall654_sensor1216c1220_electric-mains_...
1,59,electric-mains_electric-combined,None,home59_hall687_sensor1506c1508_electric-mains_...
2,61,electric-mains_electric-combined,None,home61_hall693_sensor1563c1569_electric-mains_...
3,61,electric-subcircuit,mains,home61_utility1109_sensor5071_electric-subcirc...
4,62,electric-mains_electric-combined,None,home62_hall705_sensor1662c1666_electric-mains_...
...,...,...,...,...
329,331,electric-mains_electric-combined,None,home331_hall2999_sensor21563c21567_electric-ma...
330,332,electric-mains_electric-combined,None,home332_hall3027_sensor21901c21903_electric-ma...
331,333,electric-mains_electric-combined,None,home333_hall3015_sensor21748c21765_electric-ma...
332,334,electric-mains_electric-combined,None,home334_hall3013_sensor21787c21791_electric-ma...


### Poprawienie pierwszej kolumny i agregacja dancyh do dziennych - przykład

In [ ]:
df = pl.read_parquet("../../Data/Parquet/room_and_appliance_sensors/sensordata/home61_kitchen695_sensor1967_electric-appliance_kettle.parquet")
df_clean = fix_parquet_with_header_data(df)
daily_kettle = aggregate_data_daily(df_clean)

In [15]:
# zaczytanie danych o domu
df_home_id=pl.read_csv('../../Data/NAV/Features_HomeID.csv')
df_home_id

homeid_1,num_males,num_females,num_residents,num_children,num_employed,num_with_higher_education,daily_standing_charge_pence,unit_charge_pence_per_kwh,n_rooms,smart_automation,hometype
i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,str,str
61,1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
61,1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
61,1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
62,1,1,2,0,2,2,36.0,16.16,7,"""Don't own""","""flat"""
62,1,1,2,0,2,2,36.0,16.16,7,"""Don't own""","""flat"""
…,…,…,…,…,…,…,…,…,…,…,…
328,0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""
328,0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""
328,0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""


### appliance_data_to_process

In [16]:
# Zmiana appliance_data_to_process na DataFrame Polars
appliance_data_to_process_pl = pl.DataFrame(appliance_data_to_process)
appliance_data_to_process_pl

home_id,electric,appliance_name,filename
i64,str,str,str
61,"""electric-appliance""","""kettle""","""home61_kitchen695_sensor1967_e…"
61,"""electric-appliance""","""dishwasher""","""home61_kitchen695_sensor1969_e…"
61,"""electric-appliance""","""microwave""","""home61_kitchen695_sensor1971_e…"
62,"""electric-appliance""","""fridgefreezer""","""home62_kitchen710_sensor1779_e…"
62,"""electric-appliance""","""kettle""","""home62_kitchen710_sensor1780_e…"
…,…,…,…
328,"""electric-appliance""","""vacuumcleaner""","""home328_hall2976_sensor31009_e…"
328,"""electric-appliance""","""kettle""","""home328_kitchen2980_sensor3100…"
328,"""electric-appliance""","""toaster""","""home328_kitchen2980_sensor3100…"


In [17]:
df_joined = appliance_data_to_process_pl.join(
    df_home_id.unique(subset=["homeid_1"]),
    left_on='home_id',
    right_on='homeid_1',
    how='left'
)
df_joined

home_id,electric,appliance_name,filename,num_males,num_females,num_residents,num_children,num_employed,num_with_higher_education,daily_standing_charge_pence,unit_charge_pence_per_kwh,n_rooms,smart_automation,hometype
i64,str,str,str,i64,i64,i64,i64,i64,i64,f64,f64,i64,str,str
61,"""electric-appliance""","""kettle""","""home61_kitchen695_sensor1967_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
61,"""electric-appliance""","""dishwasher""","""home61_kitchen695_sensor1969_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
61,"""electric-appliance""","""microwave""","""home61_kitchen695_sensor1971_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
62,"""electric-appliance""","""fridgefreezer""","""home62_kitchen710_sensor1779_e…",1,1,2,0,2,2,36.0,16.16,7,"""Don't own""","""flat"""
62,"""electric-appliance""","""kettle""","""home62_kitchen710_sensor1780_e…",1,1,2,0,2,2,36.0,16.16,7,"""Don't own""","""flat"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
328,"""electric-appliance""","""vacuumcleaner""","""home328_hall2976_sensor31009_e…",0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""
328,"""electric-appliance""","""kettle""","""home328_kitchen2980_sensor3100…",0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""
328,"""electric-appliance""","""toaster""","""home328_kitchen2980_sensor3100…",0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""


In [ ]:
df_all = []

# Iterujemy po unikalnych wierszach z potrzebnymi informacjami
for row in df_joined.iter_rows(named=True):
    home_id = row["home_id"]
    filename = row["filename"]
    device = row["appliance_name"]

    path_file = Path(f"../../Data/Parquet/room_and_appliance_sensors/sensordata/{filename}")
    # print(path_file)

    try:
        df = pl.read_parquet(path_file)
    
        if df.is_empty():
            print(f"POMINIĘTO pusty plik: {path_file}")
            continue

        df_clean = fix_parquet_with_header_data(df)
        daily_data = aggregate_data_daily(df_clean)

        df_tmp = daily_data.with_columns([
            pl.lit(home_id).alias("home_id"),
            pl.lit(device).alias("device")
        ])

        df_all.append(df_tmp)

    except Exception as e:        
        print(f"Błąd przy pliku {path_file}: {e}")
        continue

# Łączenie wszystkich przetworzonych ramek
df_all = pl.concat(df_all)

POMINIĘTO pusty plik: ../../Data/Parquet/room_and_appliance_sensors/sensordata/home139_hall1313_sensor14580_electric-appliance_vacuumcleaner.parquet
POMINIĘTO pusty plik: ../../Data/Parquet/room_and_appliance_sensors/sensordata/home328_hall2976_sensor31009_electric-appliance_vacuumcleaner.parquet


In [34]:
df_all

date,daily_sum,home_id,device
date,i64,i32,str
2016-11-17,11763,61,"""kettle"""
2016-11-18,14256,61,"""kettle"""
2016-11-19,22827,61,"""kettle"""
2016-11-20,25511,61,"""kettle"""
2016-11-21,19826,61,"""kettle"""
…,…,…,…
2018-05-26,3776,328,"""microwave"""
2018-05-27,0,328,"""microwave"""
2018-05-28,2662,328,"""microwave"""


In [ ]:
# df_joined.filter(pl.col('home_id') == 328)['filename'][-1]
# path_file = Path(f"../../Data/Parquet/room_and_appliance_sensors/sensordata/{df_joined.filter(pl.col('home_id') == 328)['filename'][-1]}")
# path_file
# clean_microwave = fix_parquet_with_header_data(pl.read_parquet(path_file))
# daily_microwave = aggregate_data_daily(clean_microwave)
# daily_microwave

date,daily_sum
date,i64
2018-05-26,3776
2018-05-27,0
2018-05-28,2662
2018-05-29,2392
2018-05-30,0


### Dojoinowanie danych o elekryczności z appliance do danych o domu

In [43]:
df_all

date,daily_sum,home_id,device
date,i64,i32,str
2016-11-17,11763,61,"""kettle"""
2016-11-18,14256,61,"""kettle"""
2016-11-19,22827,61,"""kettle"""
2016-11-20,25511,61,"""kettle"""
2016-11-21,19826,61,"""kettle"""
…,…,…,…
2018-05-26,3776,328,"""microwave"""
2018-05-27,0,328,"""microwave"""
2018-05-28,2662,328,"""microwave"""


In [ ]:
df_all = df_all.with_columns([
    pl.col("device").str.strip_chars().str.to_lowercase()
])

df_joined = df_joined.with_columns([
    pl.col("appliance_name").str.strip_chars().str.to_lowercase()
])

df_panel_long = df_all.join(
    df_joined,
    left_on=["home_id", "device"],
    right_on=["home_id", "appliance_name"],
    how="left"
)

In [36]:
df_panel_long

date,daily_sum,home_id,device,electric,filename,num_males,num_females,num_residents,num_children,num_employed,num_with_higher_education,daily_standing_charge_pence,unit_charge_pence_per_kwh,n_rooms,smart_automation,hometype
date,i64,i32,str,str,str,i64,i64,i64,i64,i64,i64,f64,f64,i64,str,str
2016-11-17,11763,61,"""kettle""","""electric-appliance""","""home61_kitchen695_sensor1967_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
2016-11-18,14256,61,"""kettle""","""electric-appliance""","""home61_kitchen695_sensor1967_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
2016-11-19,22827,61,"""kettle""","""electric-appliance""","""home61_kitchen695_sensor1967_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
2016-11-20,25511,61,"""kettle""","""electric-appliance""","""home61_kitchen695_sensor1967_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
2016-11-21,19826,61,"""kettle""","""electric-appliance""","""home61_kitchen695_sensor1967_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2018-05-26,3776,328,"""microwave""","""electric-appliance""","""home328_kitchen2980_sensor3101…",0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""
2018-05-27,0,328,"""microwave""","""electric-appliance""","""home328_kitchen2980_sensor3101…",0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""
2018-05-28,2662,328,"""microwave""","""electric-appliance""","""home328_kitchen2980_sensor3101…",0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""


In [39]:
df_panel_long.write_parquet("../../Data/ML_DF/data_frame_panel_long.parquet")

In [40]:
pivot_df = df_panel_long.pivot(
    values="daily_sum",
    index=["date",'home_id'],
    columns="device",
    aggregate_function='first'  # or "sum" if you expect multiple entries per date/device
)

/var/folders/rh/w6cl6rd95vj2g4s03scz_4fc0000gn/T/ipykernel_68391/2203284142.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  pivot_df = df_panel_long.pivot(


In [121]:
df_home_id.join(
    pivot_df,
    left_on='homeid_1',
    right_on='home_id',
    how='left')

homeid_1,num_males,num_females,num_residents,num_children,num_employed,num_with_higher_education,daily_standing_charge_pence,unit_charge_pence_per_kwh,n_rooms,smart_automation,hometype,date,kettle,dishwasher,microwave,fridgefreezer,washingmachine,vacuumcleaner,washingmachinetumbledrier,other,toaster,freezer,fridge,dehumidifier,electricheater,tumbledrier,grill,appliance,electricoven
i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,str,str,date,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
61,1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow""",2016-11-17,11763,2969,342,null,null,null,null,null,null,null,null,null,null,null,null,null,null
61,1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow""",2016-11-18,14256,14482,9147,null,null,null,null,null,null,null,null,null,null,null,null,null,null
61,1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow""",2016-11-19,22827,7705,4858,null,null,null,null,null,null,null,null,null,null,null,null,null,null
61,1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow""",2016-11-20,25511,20033,3333,null,null,null,null,null,null,null,null,null,null,null,null,null,null
61,1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow""",2016-11-21,19826,133,4,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
328,0,1,1,0,0,1,null,null,6,"""Don't own""","""flat""",2018-05-26,8791,null,3776,20097,null,null,null,null,0,null,null,null,null,null,null,null,null
328,0,1,1,0,0,1,null,null,6,"""Don't own""","""flat""",2018-05-27,12940,null,0,16200,null,null,null,null,0,null,null,null,null,null,null,null,null
328,0,1,1,0,0,1,null,null,6,"""Don't own""","""flat""",2018-05-28,2934,null,2662,18679,null,null,null,null,0,null,null,null,null,null,null,null,null


### household_sensors_data_to_process

In [45]:
# Zmiana appliance_data_to_process na DataFrame Polars
household_sensors_data_to_process = pl.DataFrame(household_sensors_data_to_process)
household_sensors_data_to_process

home_id,electric,subcircuit,filename
i64,str,str,str
47,"""electric-mains_electric-combin…",null,"""home47_hall654_sensor1216c1220…"
59,"""electric-mains_electric-combin…",null,"""home59_hall687_sensor1506c1508…"
61,"""electric-mains_electric-combin…",null,"""home61_hall693_sensor1563c1569…"
61,"""electric-subcircuit""","""mains""","""home61_utility1109_sensor5071_…"
62,"""electric-mains_electric-combin…",null,"""home62_hall705_sensor1662c1666…"
…,…,…,…
331,"""electric-mains_electric-combin…",null,"""home331_hall2999_sensor21563c2…"
332,"""electric-mains_electric-combin…",null,"""home332_hall3027_sensor21901c2…"
333,"""electric-mains_electric-combin…",null,"""home333_hall3015_sensor21748c2…"


In [50]:
df_all_sensors = []

# Iterujemy po unikalnych wierszach z potrzebnymi informacjami
for row in household_sensors_data_to_process.iter_rows(named=True):
    home_id = row["home_id"]
    filename = row["filename"]
    subcircuit = row["subcircuit"]

    path_file = Path(f"../../Data/Parquet/household_sensors/sensordata/{filename}")
    # print(path_file)

    try:
        df = pl.read_parquet(path_file)
    
        if df.is_empty():
            print(f"POMINIĘTO pusty plik: {path_file}")
            continue

        df_clean = fix_parquet_with_header_data(df)
        daily_data = aggregate_data_daily(df_clean)

        df_tmp = daily_data.with_columns([
            pl.lit(home_id).alias("home_id"),
            pl.lit(subcircuit).alias("subcircuit")
        ])

        df_all_sensors.append(df_tmp)

    except Exception as e:      
        print(f"Błąd przy pliku {path_file}: {e}")
        continue

# Łączenie wszystkich przetworzonych ramek
df_all_sensors = pl.concat(df_all_sensors)

SchemaError: type String is incompatible with expected type Null